In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import os, pickle
from sklearn.linear_model import LogisticRegression
from  sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.svm import LinearSVC


In [2]:
import pandas as pd

# Cargamos el archivo excel 
excel_path = 'Rest_Mex_2022.xlsx'
df = pd.read_excel(excel_path)
df.head()


,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel


In [ ]:
import pandas as pd
import spacy
import os, pickle

nlp = spacy.load('es_core_news_sm')

# Función para lematizar el texto
def lematizar_texto(texto):
    # Convertir la entrada a cadena si no lo es
    texto = str(texto) if not isinstance(texto, str) else texto
    doc = nlp(texto)
    return ' '.join([token.lemma_ for token in doc])

# Cargar tus datos
df = pd.read_excel('Rest_Mex_2022.xlsx')

# Lematización a la columna 'Opinion'
df['Opinion_Lematizada'] = df['Opinion'].apply(lematizar_texto)

# DataFrame solo con las columnas Opinión y Polaridad
df_reducido = df[['Opinion_Lematizada', 'Polarity']]

# Guardar el nuevo DataFrame en un archivo pickle
df_reducido.to_pickle('lematizado_polaridad.pkl')


In [4]:
df_reducido = pd.read_pickle('lematizado_polaridad.pkl')
df_reducido.head()


,Opinion_Lematizada,Polarity
0,"piensir dos vez antes de ir a este hotel , tú ...",1
1,cuatro de yo ir recientemente a Eddie's Place ...,1
2,seguir corto y simple : limpieza \n - bad . te...,1
3,al reservar uno hotel con multipropiedad Mayan...,1
4,"no perder su tiempo ni dinero , venir porque t...",1


In [5]:
df_reducido.to_csv('noticiaPolaridad.csv', index=False)

In [6]:
df = pd.read_csv('noticiaPolaridad.csv')
df.head()

,Opinion_Lematizada,Polarity
0,"piensir dos vez antes de ir a este hotel , tú ...",1
1,cuatro de yo ir recientemente a Eddie's Place ...,1
2,seguir corto y simple : limpieza \n - bad . te...,1
3,al reservar uno hotel con multipropiedad Mayan...,1
4,"no perder su tiempo ni dinero , venir porque t...",1


In [7]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('noticiaPolaridad.csv')

# 'Opinion_Lematizada' es el texto y 'Polarity' es la etiqueta
X = df['Opinion_Lematizada']
y = df['Polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [8]:
print(X_train.head())
print(y_train.head())
print(X_test.head())
print(y_test.head())

18594    también ahora , abierto solo con comida para l...
19690    desde que llegar hotel personal estar mucho at...
26183    excelente servicio poner súper tazón todo part...
26029    excelente el desayuno mucho abundante gran var...
16566    tener masaje minuto por fin pareja Katy , mani...
Name: Opinion_Lematizada, dtype: object
18594    5
19690    5
26183    5
26029    5
16566    5
Name: Polarity, dtype: int64
9730       cena ser fenomenal , como siempre , servicio...
7442     yo encaminar Puerto Vallarta por primero vez c...
25844    gran lugar para comer ! Camarón azul , pargo ,...
9565     buen comida , abundante , precio regular , sol...
718      cenar dos vez períodir dos semana noviembre no...
Name: Opinion_Lematizada, dtype: object
9730     5
7442     4
25844    5
9565     5
718      2
Name: Polarity, dtype: int64


In [9]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [16]:
feature_names = vectorizer.get_feature_names_out()
print(feature_names[2950:2960])

['articular' 'articulo' 'artifact' 'artificial' 'artificioso' 'artificiós'
 'artilugio' 'artimaña' 'artir' 'artista']


In [21]:
print(X_train_tfidf.shape)
print(X_train_tfidf.toarray()[2950:2960, 2950:2960]) 


(24169, 37058)
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.22299311]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.       

In [35]:
primer_documento = X_train.iloc[0]
print("Texto del primer documento:", primer_documento)


Texto del primer documento: también ahora , abierto solo con comida para llevar domicilio mejór mejór : mismo sabor , comida caliente servicio amable ... como siempre ! ! ! servicio para llevar , como prometido , respetar % descuento ! con Sabrosa Italia valer pena quedar él casa ! Gracias esperar que todo pasar mucho pronto , tener gana visitar hermoso lugar , verdadero rincón Italia México !


In [19]:
doc_id = 0  # Índice del primer documento en X_train
feature_names = vectorizer.get_feature_names_out()
doc_tfidf = X_train_tfidf[doc_id].toarray().flatten()
tfidf_pairs = zip(feature_names, doc_tfidf)
print(sorted(tfidf_pairs, key=lambda x: x[1], reverse=True)[:10])  # Imprime las 10 características con mayor TF-IDF


[('mejór', 0.5402036802128284), ('italia', 0.3900660402775469), ('domicilio', 0.2160945234700174), ('sabrosa', 0.20556377180439542), ('prometido', 0.19891666271390224), ('rincón', 0.1916552764820894), ('llevar', 0.18761546276072794), ('respetar', 0.17397151680752948), ('gana', 0.14470073427888688), ('descuento', 0.14428847118960025)]


In [45]:
# Modelo LinealSVC + TF-IDF 
clf_polarity = LinearSVC(  max_iter=30000, C=1.0, verbose=1)
clf_polarity.fit(X_train_tfidf, y_train)

# conjunto de prueba
y_pred = clf_polarity.predict(X_test_tfidf)

# predicciones
print(y_pred)

# precisión
print("Accuracy:", accuracy_score(y_test, y_pred))

#  matriz de confusión
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

#  informe de clasificación
print("Classification Report:\n", classification_report(y_test, y_pred))

# Realizamos validación cruzada
print('Realizando validación cruzada...')
cv_results = cross_validate(clf_polarity, X_train_tfidf, y_train, cv=5, scoring='f1_macro', verbose=3, n_jobs=-1)

# Imprimimos los resultados de la validación cruzada
print(cv_results)

# Promedio de f1_macro en validación cruzada
print('Media de f1_macro en validación cruzada: ', np.mean(cv_results['test_score']))

[LibLinear][5 5 5 ... 5 5 5]
Accuracy: 0.7034585470792653
Confusion Matrix:
 [[  47   21   19   10    7]
 [  22   36   47   23   17]
 [  20   40  171  127   64]
 [  11   24  155  474  499]
 [  12   23  127  524 3523]]
Classification Report:
               precision    recall  f1-score   support

           1       0.42      0.45      0.44       104
           2       0.25      0.25      0.25       145
           3       0.33      0.41      0.36       422
           4       0.41      0.41      0.41      1163
           5       0.86      0.84      0.85      4209

    accuracy                           0.70      6043
   macro avg       0.45      0.47      0.46      6043
weighted avg       0.71      0.70      0.71      6043

Realizando validación cruzada...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    8.2s remaining:   12.4s


{'fit_time': array([8.2220366 , 8.31764627, 8.44222021, 8.14252424, 5.43173194]), 'score_time': array([0.01700115, 0.01996469, 0.02297664, 0.01851106, 0.01900005]), 'test_score': array([0.43531073, 0.43400835, 0.44405049, 0.43477322, 0.44997009])}
Media de f1_macro en validación cruzada:  0.43962257536900984


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   13.7s finished


In [10]:
# Modelo REGRESIÓN LOGÍSITCA + TF-IDF 
clf_polarity = LogisticRegression(max_iter=30000, C=1.0, verbose=1)
clf_polarity.fit(X_train_tfidf, y_train)

y_pred = clf_polarity.predict(X_test_tfidf)

print(y_pred)

print("Accuracy:", accuracy_score(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

print("Classification Report:\n", classification_report(y_test, y_pred))

print('Realizando validación cruzada...')
cv_results = cross_validate(clf_polarity, X_train_tfidf, y_train, cv=5, scoring='f1_macro', verbose=3, n_jobs=-1)

print(cv_results)
print('Media de f1_macro en validación cruzada: ', np.mean(cv_results['test_score']))

c:\Users\marcos\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


[5 4 5 ... 5 4 5]
Accuracy: 0.6496773125930829
Confusion Matrix:
 [[  51   23   22    8    0]
 [  29   45   45   19    7]
 [  38   52  192  117   23]
 [  21   40  222  596  284]
 [  33   38  173  923 3042]]
Classification Report:
               precision    recall  f1-score   support

           1       0.30      0.49      0.37       104
           2       0.23      0.31      0.26       145
           3       0.29      0.45      0.36       422
           4       0.36      0.51      0.42      1163
           5       0.91      0.72      0.80      4209

    accuracy                           0.65      6043
   macro avg       0.42      0.50      0.44      6043
weighted avg       0.73      0.65      0.68      6043

Realizando validación cruzada...


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   57.2s remaining:  1.4min


{'fit_time': array([37.69304633, 37.79300094, 35.10799766, 35.03504848, 21.41998935]), 'score_time': array([0.02499986, 0.0279994 , 0.03199935, 0.01999617, 0.02000642]), 'test_score': array([0.45870917, 0.44866661, 0.44730718, 0.44452941, 0.46830655])}
Media de f1_macro en validación cruzada:  0.45350378604720143


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.3min finished
